In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# 加载预训练的 GPT 模型和分词器
model_name = 'gpt2-medium'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# 输入食材列表
ingredients = "1 cup flour, 1 egg, 1 cup milk, 1/2 teaspoon salt"

# 将食材列表编码成输入文本
input_text = f"Ingredients: {ingredients}. Recipe:"
tokenizer.pad_token = tokenizer.eos_token
# 将输入文本编码成模型输入的 token IDs
input_ids = tokenizer.encode(input_text, max_length = 50, padding = "max_length", truncation = True, return_tensors='pt')
print('token len', (input_ids))
print('text len',len(input_text))
# 生成食谱文本
output = model.generate(input_ids,do_sample = True, max_length=100, top_k = 10, temperature=0.8)

# 解码生成的食谱文本
recipe_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(recipe_text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


token len tensor([[41222,    25,   352,  6508, 10601,    11,   352,  5935,    11,   352,
          6508,  7545,    11,   352,    14,    17, 22326,  8268,    13, 26694,
            25, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256]])
text len 71
Ingredients: 1 cup flour, 1 egg, 1 cup milk, 1/2 teaspoon salt. Recipe:I have been thinking about the "big picture" for the past week.

I was talking to a friend of mine about the recent news of a new study that shows Americans' willingness to pay more for health insurance has dropped dramatically over the past


In [1]:
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.9/801.9 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 67.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-man

In [2]:
from torch.utils.data import Dataset , DataLoader
import pandas as pd
from transformers import  GPT2LMHeadModel, GPT2Tokenizer,AdamW
import json
class ReceipeDataset(Dataset):
    def __init__(self,datas, tokenizer):
        self.tokenizer = tokenizer
        self.ingredients = datas['ingredients'].values
        self.directions = datas['directions'].values
    def __len__(self):
        return len(self.ingredients)

    def __getitem__(self,index):
        ingredient = self.ingredients[index]
        direction = self.directions[index]
        ingredient_token = self.tokenizer.encode(ingredient , max_length = 100 , padding = "max_length" , truncation = True, return_tensors = "pt").reshape(-1)
        direction_token = self.tokenizer.encode(direction , max_length = 100 , padding = "max_length" , truncation = True, return_tensors = "pt").reshape(-1)
        return ingredient_token,direction_token


if __name__ == '__main__':

    with open('test.json', 'r') as file:
        data = json.load(file)

    df = pd.read_csv("test.csv")
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
    tokenizer.pad_token = tokenizer.eos_token
    dset = ReceipeDataset(df,tokenizer)
    title = next(iter(DataLoader(dset , batch_size = 1,shuffle = True)))
    print(title)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

[tensor([[41222,    25,   352, 18360,    13,  2323, 12023,   352,    14,    17,
           269,    13, 48427, 21670,   352,   357,  1415,   352,    14,    17,
         15649,  2014,   460, 23972, 20720,   513,    14,    19,   269,    13,
         38573, 11464,   513,   309, 24145,    13, 20720,  4077, 13385,   352,
            14,    17,   269,    13, 26790, 22259, 25776,  1083,   352,   352,
            14,    17, 23053,    13, 32728, 11913,   352,    14,    17, 23053,
            13, 17644,  8268,   352,    14,    17, 23053,    13,  8268,   352,
           269,    13,  1660,   352,   269,    13, 37624,   327, 44937,  9891,
            13, 26694,    25, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256]]), tensor([[41222,    25,   352, 18360,    13,  2323, 12023,   352,    14,    17,
           269,    13, 48427, 21670,   352,   357,  1415,   352,    14,    17,
         15649,  2014,   460, 23972, 20720,   51

In [3]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader
# from ReceipeDataset import ReceipeDataset
from transformers import AdamW

gpt2 = GPT2LMHeadModel.from_pretrained("gpt2-medium", cache_dir="./cache/gpt2model")
class DataModule(pl.LightningDataModule):
    def __init__(self,x_train,x_test, tokenizer):
        super().__init__()
        self.train = ReceipeDataset(x_train,tokenizer)
        self.test = ReceipeDataset(x_test,tokenizer)
        self.val = ReceipeDataset(x_test,tokenizer)

    def train_dataloader(self):
        return DataLoader(self.train , batch_size = 1 , shuffle = True)
    def test_dataloader(self):
        return DataLoader(self.test , batch_size = 1 , shuffle = False)
    def val_dataloader(self):
        return DataLoader(self.val , batch_size = 1 , shuffle = False)

class Model(pl.LightningModule):
    def __init__(self):
        super().__init__()
        gpt2.train()
        self.model = gpt2

    def forward(self,x):
        # print(x)
        return self.model(x[1] , labels = x[1])

    def configure_optimizers(self):
        return AdamW(self.parameters(), 1e-4)

    def training_step(self,batch,batch_idx):
        x= batch
        output = self(x)
        return output.loss

    def test_step(self,batch,batch_idx):
        x= batch
        output = self(x)
        return output.loss

    def validation_step(self,batch,batch_idx):
        x= batch
        output = self(x)
        return output.loss

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
from transformers import  GPT2LMHeadModel, GPT2Tokenizer,AdamW
import pandas as pd
from torch.utils.data import Dataset , DataLoader
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
from pytorch_lightning import Trainer
# from model import Model,DataModule


tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium", cache_dir="./cache/gpt2token")
tokenizer.pad_token = tokenizer.eos_token

df = pd.read_csv("test.csv")
x_train , x_test = train_test_split(df, test_size = 0.3 , random_state = 42)


model = Model()
module = DataModule(x_train , x_test,tokenizer)
trainer = Trainer(max_epochs = 8)
trainer.gpus = 1

trainer.fit(model,module)



tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=8` reached.


In [5]:
from google.colab import drive
drive.mount('/gdrive')

%cd '/gdrive/My Drive/ai-finalproject'

Mounted at /gdrive
/gdrive/My Drive/ai-finalproject


In [6]:
%cp /content/lightning_logs/version_0/checkpoints/epoch=7-step=5600.ckpt test.pt


In [7]:
import torch

modell = Model.load_from_checkpoint("test.pt")
gpt2.state_dict = modell.state_dict
raw_text = ["Ingredients: milk. Recipe:" ,"Ingredients: 1 box powdered sugar 8 oz. Recipe:"  , "Ingredients: 1 box yellow cake mix 2 eggs 1/3 c. soft margarine 1 can Eagle Brand milk 1 c. pecans 1/2 c. Bits 'O Brickle. Recipe:" , "Ingredients: 1 can Spanish rice 1 can corned beef 1 diced onion 1/2 c. grated Cheddar cheese. Recipe:"]
output_text = []

gpt2.to("cpu")
for x in raw_text:
    prompt = tokenizer.encode(x , return_tensors = "pt")
    prompt.to("cpu")
    output = gpt2.generate(prompt,do_sample = True, max_length = 100,top_k = 10, temperature = 0.8)
    output_text.append(tokenizer.decode(output[0] , skip_special_tokens = True))

for t in output_text:
  print(t)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Ingredients: milk. Recipe: Put milk in mixer bowl. Add sugar and beat well. Stir in flour and baking soda. Add vanilla. Beat until smooth. Pour into 2 greased loaf pans. Bake at 350\u00b0 for 40 minutes. Freezes well.
Ingredients: 1 box powdered sugar 8 oz. Recipe: Mix together box vanilla wafers and sugar. Roll into balls and bake as needed. Cool down.
Ingredients: 1 box yellow cake mix 2 eggs 1/3 c. soft margarine 1 can Eagle Brand milk 1 c. pecans 1/2 c. Bits 'O Brickle. Recipe: Mix together with a fork: cake mix 1 egg and margarine. Spread on ungreased 11 x 13-inch cake pan. Eagle Brand milk and pecans should be close to room temperature. Pour over cake. Bake 40 minutes at 350\u00b0. Cool 20 minutes
Ingredients: 1 can Spanish rice 1 can corned beef 1 diced onion 1/2 c. grated Cheddar cheese. Recipe: Mix all ingredients in a large bowl. Form into ball and roll in chopped pecans. Wrap in foil and refrigerate overnight. Bake at 400\u00b0 for 20 to 25 minutes or until golden brown.
